In [232]:
import pandas as pd  
LTI_data = pd.read_excel(r"C:\Users\vijyoti\Downloads\Case Study - Deep Dive Analysis.xlsx")
LTI_data.head()

,Zone,Region,Manufacturer,Brand,Subbrand,Item,PackSize,Packtype,month,Value Offtake(000 Rs)
0,East,Rural,GLAXOSMITHKLINE,HORLICKS,HORLICKS LITE,HORLICKS LITE 500 GMS REFILL PACK,500 GMS,REFILL PACK,2019-01-01,4883.050
1,East,Rural,GLAXOSMITHKLINE,HORLICKS,WOMENS HORLICKS,WOMENS HORLICKS 500 GMS REFILL PACK,500 GMS,REFILL PACK,2019-01-01,4460.013
2,East,Rural,GLAXOSMITHKLINE,BOOST,BOOST SPORTS,BOOST SPORTS 500 GMS REFILL PACK,500 GMS,REFILL PACK,2019-01-01,3230.254
3,East,Rural,GLAXOSMITHKLINE,HORLICKS,HORLICKS HEALTH AND NUTRITION,HORLICKS HEALTH AND NUTRITION 500 GMS REFILL PACK,500 GMS,REFILL PACK,2019-01-01,4548.961
4,East,Rural,GLAXOSMITHKLINE,BOOST,BOOST HEALTH AND ENERGY,BOOST HEALTH AND ENERGYÂ 500 GMS REFILL PACK,500 GMS,REFILL PACK,2019-01-01,2993.130


In [233]:
LTI_data.columns

Index(['Zone', 'Region', 'Manufacturer', 'Brand', 'Subbrand', 'Item',
       'PackSize', 'Packtype', 'month', 'Value Offtake(000 Rs)'],
      dtype='object')

In [234]:
print(LTI_data.shape)
print(LTI_data.isnull().sum())

(1280, 10)
Zone                     0
Region                   0
Manufacturer             0
Brand                    0
Subbrand                 0
Item                     0
PackSize                 0
Packtype                 0
month                    0
Value Offtake(000 Rs)    0
dtype: int64


In [235]:
#Converting Month column into datetimes series
LTI_data["datamonth"] = pd.to_datetime(LTI_data.month)

In [236]:
LTI_data.head(2)

,Zone,Region,Manufacturer,Brand,Subbrand,Item,PackSize,Packtype,month,Value Offtake(000 Rs),datamonth
0,East,Rural,GLAXOSMITHKLINE,HORLICKS,HORLICKS LITE,HORLICKS LITE 500 GMS REFILL PACK,500 GMS,REFILL PACK,2019-01-01,4883.050,2019-01-01
1,East,Rural,GLAXOSMITHKLINE,HORLICKS,WOMENS HORLICKS,WOMENS HORLICKS 500 GMS REFILL PACK,500 GMS,REFILL PACK,2019-01-01,4460.013,2019-01-01


In [237]:
#Setting Index for month wise filter 
LTI_data.set_index("datamonth",inplace=True)

In [238]:
LTI_data.head(2)

,Zone,Region,Manufacturer,Brand,Subbrand,Item,PackSize,Packtype,month,Value Offtake(000 Rs)
datamonth,,,,,,,,,,
2019-01-01,East,Rural,GLAXOSMITHKLINE,HORLICKS,HORLICKS LITE,HORLICKS LITE 500 GMS REFILL PACK,500 GMS,REFILL PACK,2019-01-01,4883.050
2019-01-01,East,Rural,GLAXOSMITHKLINE,HORLICKS,WOMENS HORLICKS,WOMENS HORLICKS 500 GMS REFILL PACK,500 GMS,REFILL PACK,2019-01-01,4460.013


##  UDF as per my requirements

In [239]:
def Deep_Dive_Analysis(Manufacturer, reference_period,target_period):
    ''' This function will take manufacturer name, target period ("MMM-YYYY") and reference period ("MMM-YYYY") and it will
        return a dataframe with manufacturer, focus area, growth rate, contribution & product.
        
        eg.  func(""GLAXOSMITHKLINE","Mar-2019","Jan-2019")
    '''
    # Creting Dataframe as per requirements 
    df = pd.DataFrame(columns=["Manufacturer","focus_area","growth_rate","contribution","product"])
    
    #changing the month fromat for example - 'Feb-2019' --->02-2019
    month_dict = {"jan":'01',"feb":"02","mar":"03","apr":"04","may":"05","jun":"06","jul":"07","aug":"08","sep":"09","oct":"10","nov":"11","dec":"12"}
    
    #Filtering data accourding to given manufacturer 
    new_data = LTI_data[LTI_data["Manufacturer"]==Manufacturer] 
    
    #Filtering data according to target-period and reference_period
    t_MY = month_dict[target_period.split("-")[0].lower()]+"-"+target_period.split("-")[1]
    r_MY = month_dict[reference_period.split("-")[0].lower()]+"-"+reference_period.split("-")[1]

    target_value_sale = new_data.loc[t_MY,"Value Offtake(000 Rs)"].sum()
    ref_value_sale = new_data.loc[r_MY,"Value Offtake(000 Rs)"].sum()
    if target_value_sale > ref_value_sale:
        return "There is no drop in the sales for a "+Manufacturer+" in the "+reference_period+" to "+target_period
    else:
        target_data = new_data.loc[t_MY,:] 
        ref_data = new_data.loc[r_MY,:]      
        
        # Finding Focus Area
        focus_areas = ["Zone","Region","Brand","Subbrand","Item"]
        for area in focus_areas:
            tar_focus_area = target_data.groupby(area).sum()  
            ref_focus_area = ref_data.groupby(area).sum()
            for indx in tar_focus_area.index:
                if tar_focus_area.loc[indx,:][0] < ref_focus_area.loc[indx,:][0]:
                    growth_rate = (tar_focus_area.loc[indx,:][0] - 
                                   ref_focus_area.loc[indx,:][0])*100/ref_focus_area.loc[indx,:][0]

                    contribution = tar_focus_area.loc[indx,:][0] * 100 /tar_focus_area.sum()[0]
                    product = growth_rate * contribution/10000  
                    #appending Value in Dataframe
                    df = df.append({"Manufacturer":Manufacturer,"focus_area":area+"-"+indx,
                               "growth_rate":str(round(growth_rate,2))+"%","contribution":str(round(contribution,2))+"%","product":round(product,2)},ignore_index=True)
        return df



##  Final Output

In [247]:
Deep_Dive_Analysis("GLAXOSMITHKLINE","JAN-2019","May-2019")

,Manufacturer,focus_area,growth_rate,contribution,product
0,GLAXOSMITHKLINE,Zone-East,-20.88%,30.9%,-0.06
1,GLAXOSMITHKLINE,Zone-North,-6.35%,35.72%,-0.02
2,GLAXOSMITHKLINE,Zone-South,-21.5%,3.15%,-0.01
3,GLAXOSMITHKLINE,Zone-West,-9.11%,30.23%,-0.03
4,GLAXOSMITHKLINE,Region-Rural,-9.94%,29.39%,-0.03
5,GLAXOSMITHKLINE,Region-Urban,-13.71%,70.61%,-0.10
6,GLAXOSMITHKLINE,Brand-BOOST,-13.33%,13.18%,-0.02
7,GLAXOSMITHKLINE,Brand-HORLICKS,-12.53%,86.82%,-0.11
8,GLAXOSMITHKLINE,Subbrand-BOOST HEALTH AND ENERGY,-19.65%,6.11%,-0.01
9,GLAXOSMITHKLINE,Subbrand-BOOST SPORTS,-7.01%,7.07%,-0.00
